In [ ]:
import requests
import snowflake.connector

con = snowflake.connector.connect(connection_name='jinjat_aws_us')
cur = con.cursor()
token_data = con._rest._token_request('ISSUE')
token_extract = token_data['data']['sessionToken']

token = f'\"{token_extract}\"'
headers = {'Authorization': f'Snowflake Token={token}'}
# Set this to the ingress endpoint URL for your service
url = 'https://e6szq-eptceco-aws-us-west.snowflakecomputing.app'

# Validate the connection.
response = requests.get(f'{url}', headers=headers)
response.text

cur.execute("select * from table(to_query('select 1'))")

In [ ]:
!pip install jupyter

In [ ]:
import sqlglot
import snowflake.connector


def transformation(param):
    pass


@transformation('default_create_table_as_iceberg')
def default_create_table_as_iceberg(expression: sqlglot.exp.Expression, args: dict):
    if isinstance(expression, sqlglot.exp.Create):
        if expression.kind == 'TABLE':
            # Apply transformation
            return expression
    return expression

con = snowflake.connector.connect(
      account='lt51601.europe-west2.gcp', 
      database='my_iceberg_jinjat', connection_name='default',
      host='localhostcomputing.com', port='8084', # points to locally running universql 
      # run on snowflake, only if there is an active warehouse
      # warehouse='sample_data(0.1) | snowflake(warehouse=already_on)',
      # try to run the query on sample data (i.e. table_ref becomes `tableref TABLESAMPLE (1000 rows)`) locally and use, Motherduck if it doesn't complete in 10 seconds, and fallback to Snowflake
      warehouse='replace_table_sample("1000 rows") | duckdb(timeout=10s),duckdb(database="md:", timeout=10m),snowflake()')
cur = con.cursor()
#con.cursor().execute("select * from \"DYNAMIC_ICEBERG_TABLE_CUSTOMER\"  limit 100").fetch_pandas_all()
con.cursor().execute("select 1  limit 100").fetch_pandas_all()


In [5]:
con.cursor().execute("select * from table(to_query('select 1'))").fetch_pandas_all()


NameError: name 'con' is not defined

In [2]:
con.cursor().execute("select C_MKTSEGMENT, count(*) from my_iceberg_jinjat.tpch_sf1.dynamic_iceberg_table_customer group by all").fetch_pandas_all()

InterfaceError: 250003 (08001): 401 Unauthorized: post https://localhostcomputing.com:8084/queries/v1/query-request?requestId=6d4b2c75-1ca1-4ca6-a3ed-af1749a5f288&clientStartTime=1727130359840&retryCount=8&retryReason=None

In [2]:
con.cursor().execute("show tables in tpch_sf1").fetch_pandas_all()

,C_MKTSEGMENT,count_star()
0,BUILDING,120568
1,AUTOMOBILE,119008
2,FURNITURE,119872
3,MACHINERY,119796
4,HOUSEHOLD,120756


In [27]:
con.cursor().execute("select count(*) from my_iceberg_jinjat.tpch_sf1.orders_iceberg_tpch_sf1000").fetch_pandas_all()

,count_star()
0,1500000000


In [28]:
con.cursor().execute("select O_ORDERDATE, count(O_CLERK) from my_iceberg_jinjat.tpch_sf1.orders_iceberg_tpch_sf1000 group by all order by 1 desc limit 1000").fetch_pandas_all()

,O_ORDERDATE,count(O_CLERK)
0,1998-08-02,623367
1,1998-08-01,623287
2,1998-07-31,623459
3,1998-07-30,623159
4,1998-07-29,623071
...,...,...
995,1995-11-11,623359
996,1995-11-10,622651
997,1995-11-09,624224
998,1995-11-08,622410
